In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
import time
import numpy as np
import plotly.express as px
from dataclasses import dataclass

torch.manual_seed(10)



input_size =5
learning_rate = 0.01
max_iters = 1000
dataset_size = 1024*32
sparsity = 0.1
batch_size = 32

iters = []
loss_value = []
val_loss_value = []


def sample_data(split = 'train'):
    if split != 'train':
        curr_data = validation_dataset
    else:
        curr_data = training_dataset
    sparsity_vector = sparsity
    a, b = curr_data.shape
    
    sparsity_vector = torch.rand(5)
    sparsity_vector = sparsity_vector.view(1, -1)
    sparsity_mat = torch.bernoulli(torch.full((a, b), 1.0) * sparsity_vector)
    
    X = curr_data * sparsity_mat
    return X, X



DEVICE = torch.device('cpu')

@dataclass
class Config:
    input_size : int = 5
    layers : int = 2

class Two_Two(nn.Module):
    def __init__(self) -> None:
        self.layers = nn.ModuleList([nn.Linear(2,2), nn.ReLU()])

class Model(nn.Module):
    def __init__(self, config : Config) -> None:
        super().__init__()
        self.config = config
        self.layers = nn.ModuleList([nn.Linear(config.input_size, 2)]+[Two_Two() for i in range(config.layers)] + [nn.Linear(2, 1)])
    
    def forward(self, idx : torch.Tensor):
        count = self.layers(idx)
        return count, (count-(idx>0).sum())**2


        

training_dataset = torch.rand(dataset_size, input_size)
validation_dataset = torch.rand(int(dataset_size/2), input_size)
output = input


model = Model(Config())


optimizer = torch.optim.Adam(model.parameters() , learning_rate, weight_decay= 1e-5)
start_time = time.time()
step_value = max_iters/20
for i in range(1, max_iters):
    X, Y = sample_data()
    logits, loss, _ = model(X, Y)

    if i%step_value ==0 :
        model.eval()
        with torch.no_grad():
            X_V,Y_V = sample_data(split = 'val')
            _, val_loss, _ = model(X_V, Y_V)
            iters.append(i)
            loss_value.append(loss.item())
            val_loss_value.append(val_loss.item())
            print(f"iter:{i} training loss: {loss.item()}, val loss: {val_loss.item()}")
        model.train()

    optimizer.zero_grad(set_to_none=True)
    #loss+= get_l1_penalty()
    loss.backward()
    optimizer.step()


end_time = time.time()
print(f"Took {end_time-start_time}s for {max_iters} epochs")

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(iters,loss_value, color='blue', label="Training")
plt.plot(iters, val_loss_value, "red", label = "validation")
plt.legend()
plt.show()

In [1]:
import torch

In [2]:
a = torch.rand(5,5)

In [5]:
(a>=0).sum()

tensor(25)

In [8]:
torch.rand(5)

tensor([0.3023, 0.3675, 0.1506, 0.4065, 0.0530])